In [1]:
# coding=UTF-8
import xlrd
import xlwt
from xlutils.copy import copy
import requests
import time
import hashlib
import json

In [2]:
##用于写入数据
def write_excel_xls(path, sheet_name, value):
    index = len(value)  # 获取需要写入数据的行数
    workbook = xlwt.Workbook()  # 新建一个工作簿
    sheet = workbook.add_sheet(sheet_name)  # 在工作簿中新建一个表格
    for i in range(0, index):
        for j in range(0, len(value[i])):
            sheet.write(i, j, value[i][j])  # 像表格中写入数据（对应的行和列）
    workbook.save(path)  # 保存工作簿
    print("xls格式表格写入数据成功！")

In [3]:
##用于增补数据
def write_excel_xls_append(path, value):
    index = len(value)  # 获取需要写入数据的行数
    workbook = xlrd.open_workbook(path)  # 打开工作簿
    sheets = workbook.sheet_names()  # 获取工作簿中的所有表格
    worksheet = workbook.sheet_by_name(sheets[0])  # 获取工作簿中所有表格中的的第一个表格
    rows_old = worksheet.nrows  # 获取表格中已存在的数据的行数
    new_workbook = copy(workbook)  # 将xlrd对象拷贝转化为xlwt对象
    new_worksheet = new_workbook.get_sheet(0)  # 获取转化后工作簿中的第一个表格
    for i in range(0, index):
        for j in range(0, len(value[i])):
            new_worksheet.write(i+rows_old, j, value[i][j])  # 追加写入数据，注意是从i+rows_old行开始写入
    new_workbook.save(path)  # 保存工作簿

## 准备工作

In [10]:
#  token可以从 数据中心 -> 我的接口 中获取
token = "9320ff98-a6be-4136-8d81-49cccf76a5bc"
encode = 'utf-8'
keyword = '浙江壹云云计算有限公司'
book_name_xls = '数据写入.xls' 
sheet_name_xls = '抓取数据写入' 
value_title = [["公司名（著作人）","软件名称","版本号","发布日期","软件简称","登记号","登记批准日期"]]
write_excel_xls(book_name_xls, sheet_name_xls, value_title)

xls格式表格写入数据成功！


In [11]:
url = "http://open.api.tianyancha.com/services/open/ipr/copyReg/2.0?keyword=农村金融&pageNum=1&pageSize=20"
headers={'Authorization': token}
response = requests.get(url, headers=headers)  #返回结果

## 结果处理  

In [12]:
m=json.loads(response.text)
if m["result"] == None:
    value = [[keyword,"无数据"],]
    write_excel_xls_append(book_name_xls, value)
    print("%s无软件著作权数据"%keyword)
else:
    total = m["result"]["total"]
    print("该关键词共有%s条软件著作权数据"%total)
    tips = int(total)
    if tips <= 20:
        for i in range(tips):
            value = [[m["result"]["items"][i]["authorNationality"],m["result"]["items"][i]["fullname"],m["result"]["items"][i]["version"],m["result"]["items"][i]["publishtime"],m["result"]["items"][i]["simplename"],m["result"]["items"][i]["regnum"],m["result"]["items"][i]["eventTime"]]]
            write_excel_xls_append(book_name_xls, value)
        print(keyword+"前"+str(tips)+"条数据写入成功")
    else:
        for i in range(20):
            value = [[m["result"]["items"][i]["authorNationality"],m["result"]["items"][i]["fullname"],m["result"]["items"][i]["version"],m["result"]["items"][i]["publishtime"],m["result"]["items"][i]["simplename"],m["result"]["items"][i]["regnum"],m["result"]["items"][i]["eventTime"]]]
            write_excel_xls_append(book_name_xls, value)
        print(keyword+"前"+str(20)+"条数据写入成功")
        
        for i in range(2,int(tips/20)+2):
            url = "http://open.api.tianyancha.com/services/open/ipr/copyReg/2.0?keyword=keyword&pageNum="+str(i)+"&pageSize=20"
            response = requests.get(url, headers=headers)  #返回结果
            m=json.loads(response.text)
            for j in range(len(m["result"]["items"])):
                value = [[m["result"]["items"][i]["authorNationality"],m["result"]["items"][i]["fullname"],m["result"]["items"][i]["version"],m["result"]["items"][i]["publishtime"],m["result"]["items"][i]["simplename"],m["result"]["items"][i]["regnum"],m["result"]["items"][i]["eventTime"]]]
                write_excel_xls_append(book_name_xls, value)
            print(keyword+"前"+str(20*(i-1)+len(m["result"]["items"]))+"条数据写入成功")

浙江壹云云计算有限公司无软件著作权数据
